# 02_RNN_LSTM_Model

Tutorial source: https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/
<br> GitHub repo: https://keras.io/examples/lstm_text_generation/

In [1]:
# Load dependencies:
import pandas as pd
import numpy as np
import os
import re

# View warnings once (or ignore)
import warnings
warnings.filterwarnings(action='once')
#warnings.filterwarnings('ignore')

# Show multiple results per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# See wider columns (for text)
pd.set_option('max_colwidth', 500)

### Loading & Pre-Processing Data

In [2]:
# To improve:
# Select artist
# Download all tracks
# & then load!

In [3]:
# Load lyric directory pre-downloaded with LyricsGenius package:
files = pd.DataFrame(os.listdir("LyricsGenius/Manowar/"), columns = ["json_file_name"])
# Exclude non json files:
files = files[files['json_file_name'].str.contains(".json")]
# Quick preview:
files.head()

,json_file_name
1,lyrics_manowar_achillesagonyandecstasyineightparts.json
2,lyrics_manowar_allmenplayon10.json
3,lyrics_manowar_allmenplayonten.json
4,lyrics_manowar_anamericantrilogy.json
5,lyrics_manowar_animals.json


In [4]:
# Create empty archive:
archive = pd.DataFrame([])
# Loop through json files in directory and append to archive:
for i in range(0, len(files)):
    # Load raw json file:
    raw_json = pd.read_json("LyricsGenius/Manowar/"+files['json_file_name'].iloc[i])
    # Load json dictionary as df:
    lyrics = pd.DataFrame(raw_json['songs'][0], index = [0])
    # Add artist:
    lyrics['artist'] = raw_json['artist'][0]
    # Drop undesired columns:
    lyrics = lyrics.drop(['image', 'year'], axis=1)
    # Append to archive:
    archive = archive.append(lyrics)
    # Reset index:
    archive = archive.reset_index()
    archive = archive.drop(['index'], axis=1)

In [5]:
# Quick preview:
archive.head()

,title,album,lyrics,artist
0,"Achilles, Agony And Ecstasy In Eight Parts",The Triumph Of Steel,"(Prelude)\n\nI HECTOR STORMS THE WALL\nSee my chariot, run to your ships\nI'll drive you back to the sea\nYou came here for gold, the wall will not hold\nThis day was promised to me\nThe Gods are my shield, my fate has been sealed\nLightning and javelins fly\nSoon many will fall, we are storming the wall\nStones fall like snow from the sky\n\nWe will pay with our glory in the fire of battle\nZeus, today is mine\nKilling all in my way like sheep and like cattle\nSmashing skulls of all who def...",Manowar
1,All Men Play On 10,Sign of the Hammer,"I made a rock'n'roll sin\nWhen I tried givin' in to\nMake money had to turn down low\nThey said, ""Why be proud, don't play so loud\nBe like us and get a sound that's real thin\nWear a polyester suit, act happy look cute\nGet a haircut and buy small gear""\nThat's when I turned to them and said\n""Hold it, right there!""\n\nWell it's more to me than just a job\nAnd while I'm playin' you won't get robbed\nNobody tells a man how to play\nIt just ain't that way hey, hey, hey\nCan you hear me say......",Manowar
2,All Men Play On Ten,Sign of the Hammer,"I made a rock n' roll sin, when I tried giving in\nTo make money had to turn down low\nThey said, ""why be proud, don't play so loud, be like us\nAnd get A sound that's real thin\nWear A polyester suit, act happy, look cute, get A haircut\nAnd buy small gear""\nThat's when I turned to them and said, hold it, right there\nWell it's more to me than just A job, and while I'm playin'\nYou won't get robbed\nNobody tells A man how to play, it just ain't that way\nHey, hey, hey, can you hear me say\n...",Manowar
3,An American Trilogy,Warriors of the World,"Oh, I wish, I was in the land of cotton\nOld times they are not forgotten\nLook away, look away, look away Dixie land\n\nOh, I wish, I was in Dixie away, away\nIn Dixie land I'll take my stand\nTo live and die in Dixie\n\nDixie land where I was born\nEarly Lord, one frosty morning\nLook away, look away, look away Dixie land\n\nGlory, glory, Hallelujah\nGlory, glory, Hallelujah\nGlory, glory, Hallelujah\nThis truth is marching on\n\nSo hush little baby, don't you cry\nYou know your daddy's bo...",Manowar
4,Animals,Sign of the Hammer,"Hear the call of the wild in us all\nIt waits for the night to fall\nI'm getting hot, I'm ready for the night\nNo holdin' back, let's ball\nI'm gonna give all you can take all night\nAnd leave you in the morning feeling right\n\nI'm an animal, there's an animal in me\nGonna set it free\nI'm an animal, there's an animal in me\nGonna set it free\n\nI've been looking, you've been watching the side\nThere's somethings you just can't hide\nOh, your getting wet, your working up a sweat\nYour hairs...",Manowar


In [6]:
# ManoWaR specific pre-processing:
# The track entitled "Father" has been recorded in multiple languages. The non-english versions below need to be removed:
archive[archive['title'].str.contains("Father -")]['title']

6        Apa (Father - Hungarian Version)
13        Baba (Father - Turkish Version)
40       Far (Father - Norwegian Version)
65         Isä (Father - Finnish Version)
85     Mon Père (Father - French Version)
92       Ojciec (Father - Polish Version)
94       Otac (Father - Croatian Version)
98       Padre (Father - Italian Version)
99       Padre (Father - Spanish Version)
100     Pai (Father - Portuguese Version)
115      Tată (Father - Romanian Version)
116    Tatko (Father - Bulgarian Version)
151       Vater (Father - German Version)
158       Πατερα (Father - Greek Version)
159         父 (Father - Japanese Version)
Name: title, dtype: object

In [7]:
# Determine indeces for non-english Father tracks (which standardised as "Father  - * version"):
father = archive[archive['title'].str.contains("Father -")].index.tolist()
# See Father index:
father
# Exclude Father index:
archive = archive.drop(father, axis=0)

[6, 13, 40, 65, 85, 92, 94, 98, 99, 100, 115, 116, 151, 158, 159]

In [8]:
# Check for additional pre-processing requirements:
archive

,title,album,lyrics,artist
0,"Achilles, Agony And Ecstasy In Eight Parts",The Triumph Of Steel,"(Prelude)\n\nI HECTOR STORMS THE WALL\nSee my chariot, run to your ships\nI'll drive you back to the sea\nYou came here for gold, the wall will not hold\nThis day was promised to me\nThe Gods are my shield, my fate has been sealed\nLightning and javelins fly\nSoon many will fall, we are storming the wall\nStones fall like snow from the sky\n\nWe will pay with our glory in the fire of battle\nZeus, today is mine\nKilling all in my way like sheep and like cattle\nSmashing skulls of all who def...",Manowar
1,All Men Play On 10,Sign of the Hammer,"I made a rock'n'roll sin\nWhen I tried givin' in to\nMake money had to turn down low\nThey said, ""Why be proud, don't play so loud\nBe like us and get a sound that's real thin\nWear a polyester suit, act happy look cute\nGet a haircut and buy small gear""\nThat's when I turned to them and said\n""Hold it, right there!""\n\nWell it's more to me than just a job\nAnd while I'm playin' you won't get robbed\nNobody tells a man how to play\nIt just ain't that way hey, hey, hey\nCan you hear me say......",Manowar
2,All Men Play On Ten,Sign of the Hammer,"I made a rock n' roll sin, when I tried giving in\nTo make money had to turn down low\nThey said, ""why be proud, don't play so loud, be like us\nAnd get A sound that's real thin\nWear A polyester suit, act happy, look cute, get A haircut\nAnd buy small gear""\nThat's when I turned to them and said, hold it, right there\nWell it's more to me than just A job, and while I'm playin'\nYou won't get robbed\nNobody tells A man how to play, it just ain't that way\nHey, hey, hey, can you hear me say\n...",Manowar
3,An American Trilogy,Warriors of the World,"Oh, I wish, I was in the land of cotton\nOld times they are not forgotten\nLook away, look away, look away Dixie land\n\nOh, I wish, I was in Dixie away, away\nIn Dixie land I'll take my stand\nTo live and die in Dixie\n\nDixie land where I was born\nEarly Lord, one frosty morning\nLook away, look away, look away Dixie land\n\nGlory, glory, Hallelujah\nGlory, glory, Hallelujah\nGlory, glory, Hallelujah\nThis truth is marching on\n\nSo hush little baby, don't you cry\nYou know your daddy's bo...",Manowar
4,Animals,Sign of the Hammer,"Hear the call of the wild in us all\nIt waits for the night to fall\nI'm getting hot, I'm ready for the night\nNo holdin' back, let's ball\nI'm gonna give all you can take all night\nAnd leave you in the morning feeling right\n\nI'm an animal, there's an animal in me\nGonna set it free\nI'm an animal, there's an animal in me\nGonna set it free\n\nI've been looking, you've been watching the side\nThere's somethings you just can't hide\nOh, your getting wet, your working up a sweat\nYour hairs...",Manowar
5,Annihilation,The Lord of Steel,Another lie\nTime to die\nAnother day\nBlown away\n\nAnother scam\nShow you who I am\nAnother day\nBlown away\n\nHand of justice\nSoon will be falling\nVengeance thine\nLord of steel\nAnnihilation\n\nThey will never win this fight\nWe have the power and the might\nWe have the sacred right\nFeel the power here tonight\n\nHand of justice\nSoon will be falling\nVengeance thine\nLord of steel\nAnnihilation\n\nLive the life we truly feel\nFighting for the lord of steel\nHeavy metal is my life\nHe...,Manowar
7,Army Of Immortals,None,Battle hymns did sound the call\nYou came to our side\nYou heard true metal-into glory\nRide\n\nYou stood beside us the false ones\nCried.-Your love is judgment\nYou gave us life\n\nYou wait in the rain-you walk\nThrough the snow.We give you\nOur blood.We want you to know\nIn our eyes you immortal-\nIn our hearts you'll live forever-\nIn our eyes you immortal-\nIn our hearts you'll live forever\n\nWe have read your letters-\nWe have heard your call\nWe were brought together-\nCause we've got...,Manowar
8,Army Of The Dead,None,Raise thy weapons on this day\nYe shall not die alone\nFight and die let Valkyries fly\nFor they 

In [9]:
# Some tracks are instrumental with lyrics entered as: [Instrumental]
# (normally the LyricsGenius package picks this up but apaprently it's not fail-safe)
archive[archive['lyrics']==("[Instrumental]")]

,title,album,lyrics,artist
36,Drums Of Doom,Fighting the World,[Instrumental],Manowar
80,March of the Heroes into Valhalla,The Final Battle I (EP),[Instrumental],Manowar
87,My Spirit Lives On,Louder than Hell,[Instrumental],Manowar
96,Overture To Odin,Gods of War,[Instrumental],Manowar
97,Overture To The Hymn Of The Immortal Warriors,Gods of War,[Instrumental],Manowar
112,Sting of the Bumblebee,Kings of Metal,[Instrumental],Manowar
130,The Heart of Steel MMXIV (Orchestral Intro Version),Kings of Metal MMXIV,[Instrumental],Manowar
133,The March,Warriors of the World,[Instrumental],Manowar
139,The Sting of the Bumblebee MMXIV,Kings of Metal MMXIV,[Instrumental],Manowar
144,Thunderpick,Sign of the Hammer,[Instrumental],Manowar


In [10]:
# Remove instrumental tracks:
instrumentals = archive[archive['lyrics']==("[Instrumental]")].index.tolist()
archive = archive.drop(instrumentals, axis=0)
# Reset index:
archive = archive.reset_index()
archive = archive.drop(['index'], axis=1)

In [11]:
# Some lyrics contain track composition information stored within parantheses or square brackets, as seen in the examples below:
archive['lyrics'][0]
archive['lyrics'][40]

"(Prelude)\n\nI HECTOR STORMS THE WALL\nSee my chariot, run to your ships\nI'll drive you back to the sea\nYou came here for gold, the wall will not hold\nThis day was promised to me\nThe Gods are my shield, my fate has been sealed\nLightning and javelins fly\nSoon many will fall, we are storming the wall\nStones fall like snow from the sky\n\nWe will pay with our glory in the fire of battle\nZeus, today is mine\nKilling all in my way like sheep and like cattle\nSmashing skulls of all who defy\nI spare not the hammer, I spare not the sword\nThis day will ring with my name\nNone have to chase me, let he who will face me\nKill me or die by the sword\n\nII THE DEATH OF PATROCLUS\nOh friend of mine, how to say goodbye?\nThis was your time, but the armor you wore was mine\nI will not rest until Hector's blood is spilled\nHis bones will all be broken, dragged across the field\nThis, dear friend, is how we'll say goodbye\nUntil we meet in the sky\n\nIII FUNERAL MARCH\n(Instrumental Part)\n\nI

"[Verse 1]\nWhen they come, I'll be ready\nI hear their voices inside\nThe stars in the Heavens are moving\nSoon, they will align\n\n[Pre-Chorus]\nThor, God of thunder\nLet me die with a sword in my hand\nRaise your hands, swords in the wind\nBrothers of metal, together again\nWith blood in our voices we ride\nWe'll fight till we win or we'll fight until we die\n\n[Chorus]\nGods of thunder, fire and rain\nCut into the earth till none shall remain\nSons of Demons, I command you to fly\nRide on from Hell into the sky\nWe'll fight until we die\n\n[Verse 1]\n[Pre-Chorus]\n[Chorus]\n\nGods of thunder, fire and rain\nCut into the earth where none shall remain\nSons of Demons, I command you to fly\nRide on from Hell into the sky\nWe'll fight until, fight until, fight until we die\n\n[Outro] (4x)\nSons of Demons, I command you to fly\nFight"

In [12]:
# remove these paranthesis and square brackets:
for i in range(0, len(archive)):
    archive['lyrics'][i] = re.sub("[\(\[].*?[\)\]]", "", archive['lyrics'][i])
# Quick preview
archive.head()

<>:3: DeprecationWarning: invalid escape sequence \(


,title,album,lyrics,artist
0,"Achilles, Agony And Ecstasy In Eight Parts",The Triumph Of Steel,"\n\nI HECTOR STORMS THE WALL\nSee my chariot, run to your ships\nI'll drive you back to the sea\nYou came here for gold, the wall will not hold\nThis day was promised to me\nThe Gods are my shield, my fate has been sealed\nLightning and javelins fly\nSoon many will fall, we are storming the wall\nStones fall like snow from the sky\n\nWe will pay with our glory in the fire of battle\nZeus, today is mine\nKilling all in my way like sheep and like cattle\nSmashing skulls of all who defy\nI spar...",Manowar
1,All Men Play On 10,Sign of the Hammer,"I made a rock'n'roll sin\nWhen I tried givin' in to\nMake money had to turn down low\nThey said, ""Why be proud, don't play so loud\nBe like us and get a sound that's real thin\nWear a polyester suit, act happy look cute\nGet a haircut and buy small gear""\nThat's when I turned to them and said\n""Hold it, right there!""\n\nWell it's more to me than just a job\nAnd while I'm playin' you won't get robbed\nNobody tells a man how to play\nIt just ain't that way hey, hey, hey\nCan you hear me say......",Manowar
2,All Men Play On Ten,Sign of the Hammer,"I made a rock n' roll sin, when I tried giving in\nTo make money had to turn down low\nThey said, ""why be proud, don't play so loud, be like us\nAnd get A sound that's real thin\nWear A polyester suit, act happy, look cute, get A haircut\nAnd buy small gear""\nThat's when I turned to them and said, hold it, right there\nWell it's more to me than just A job, and while I'm playin'\nYou won't get robbed\nNobody tells A man how to play, it just ain't that way\nHey, hey, hey, can you hear me say\n...",Manowar
3,An American Trilogy,Warriors of the World,"Oh, I wish, I was in the land of cotton\nOld times they are not forgotten\nLook away, look away, look away Dixie land\n\nOh, I wish, I was in Dixie away, away\nIn Dixie land I'll take my stand\nTo live and die in Dixie\n\nDixie land where I was born\nEarly Lord, one frosty morning\nLook away, look away, look away Dixie land\n\nGlory, glory, Hallelujah\nGlory, glory, Hallelujah\nGlory, glory, Hallelujah\nThis truth is marching on\n\nSo hush little baby, don't you cry\nYou know your daddy's bo...",Manowar
4,Animals,Sign of the Hammer,"Hear the call of the wild in us all\nIt waits for the night to fall\nI'm getting hot, I'm ready for the night\nNo holdin' back, let's ball\nI'm gonna give all you can take all night\nAnd leave you in the morning feeling right\n\nI'm an animal, there's an animal in me\nGonna set it free\nI'm an animal, there's an animal in me\nGonna set it free\n\nI've been looking, you've been watching the side\nThere's somethings you just can't hide\nOh, your getting wet, your working up a sweat\nYour hairs...",Manowar


In [16]:
# In this example we will be working on a Word-level model, therefore we need to remove the seperators (\n) from the lyrics:
for i in range(0, len(archive)):
    archive['lyrics'][i] = re.sub("\n", " ", archive['lyrics'][i])
# Quick preview
archive.head()

,title,album,lyrics,artist
0,"Achilles, Agony And Ecstasy In Eight Parts",The Triumph Of Steel,"I HECTOR STORMS THE WALL See my chariot, run to your ships I'll drive you back to the sea You came here for gold, the wall will not hold This day was promised to me The Gods are my shield, my fate has been sealed Lightning and javelins fly Soon many will fall, we are storming the wall Stones fall like snow from the sky We will pay with our glory in the fire of battle Zeus, today is mine Killing all in my way like sheep and like cattle Smashing skulls of all who defy I spare not the hammer...",Manowar
1,All Men Play On 10,Sign of the Hammer,"I made a rock'n'roll sin When I tried givin' in to Make money had to turn down low They said, ""Why be proud, don't play so loud Be like us and get a sound that's real thin Wear a polyester suit, act happy look cute Get a haircut and buy small gear"" That's when I turned to them and said ""Hold it, right there!"" Well it's more to me than just a job And while I'm playin' you won't get robbed Nobody tells a man how to play It just ain't that way hey, hey, hey Can you hear me say... All men play...",Manowar
2,All Men Play On Ten,Sign of the Hammer,"I made a rock n' roll sin, when I tried giving in To make money had to turn down low They said, ""why be proud, don't play so loud, be like us And get A sound that's real thin Wear A polyester suit, act happy, look cute, get A haircut And buy small gear"" That's when I turned to them and said, hold it, right there Well it's more to me than just A job, and while I'm playin' You won't get robbed Nobody tells A man how to play, it just ain't that way Hey, hey, hey, can you hear me say All men pla...",Manowar
3,An American Trilogy,Warriors of the World,"Oh, I wish, I was in the land of cotton Old times they are not forgotten Look away, look away, look away Dixie land Oh, I wish, I was in Dixie away, away In Dixie land I'll take my stand To live and die in Dixie Dixie land where I was born Early Lord, one frosty morning Look away, look away, look away Dixie land Glory, glory, Hallelujah Glory, glory, Hallelujah Glory, glory, Hallelujah This truth is marching on So hush little baby, don't you cry You know your daddy's bound to die All my ...",Manowar
4,Animals,Sign of the Hammer,"Hear the call of the wild in us all It waits for the night to fall I'm getting hot, I'm ready for the night No holdin' back, let's ball I'm gonna give all you can take all night And leave you in the morning feeling right I'm an animal, there's an animal in me Gonna set it free I'm an animal, there's an animal in me Gonna set it free I've been looking, you've been watching the side There's somethings you just can't hide Oh, your getting wet, your working up a sweat Your hairs standing up on...",Manowar


### Character-level Model

In [ ]:
'''
#Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

In [28]:
manowar = archive['lyrics'].str.cat(sep=' ').lower().replace("\"","")

In [29]:
text = manowar
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

corpus length: 161773
total chars: 42


In [30]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

nb sequences: 53911
Vectorization...


In [31]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...
Instructions for updating:
Colocations handled automatically by placer.


In [32]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [35]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    #for diversity in [0.2, 0.5, 1.0, 1.2]:
    for diversity in [0.2, 0.5]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [36]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [38]:
model.fit(x, y,
          batch_size=128,
          epochs=100,
          callbacks=[print_callback])

Epoch 1/100
53911/53911 [==============================] - 36s 676us/step - loss: 1.7533

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "me when winter skies are gray and i wait"
me when winter skies are gray and i wait we brothers the will deat steel steel steel  i mast like the steel  the steel steel stell die i will die i will die i camme the steel steel side the will death a stall deat like the steel steel  i mast like the will death i mastle the steel with all steel  i mast lise the steel the steel side  will die with with with with heir my steel deat steel steel deat speel steel steel wind a stree  i mad t
----- diversity: 0.5
----- Generating with seed: "me when winter skies are gray and i wait"
me when winter skies are gray and i wait steel steel  the deat my steel  time fight the will die stall die i crust as mast low the god of the steel  i the fore we steel i our speel sky we will blood whin spere crowd a macole the wind on the steel d